# Generation of performance files via an XGBOOST model (pickle file)

In [1]:
import os
import sys, getopt
import pandas as pd
import numpy as np
import xgboost
from sklearn.base import TransformerMixin
from sklearn.tree import DecisionTreeClassifier

from sklearn import model_selection
from sklearn.metrics import accuracy_score
from swat import *
from swat.render import render_html
from pprint import pprint
from matplotlib import pyplot as plt
import pandas as pd
import sys
%matplotlib inline

In [2]:
datadir = os.getcwd() + '/'

# This part of parameter parsing is not working properly
#print('Number of arguments:', len(sys.argv), 'arguments.')
#print('Argument List:', str(sys.argv))
#print(getArgument("VarA", "NULL"))   

def main(argv):
    uuid = ''
    arg = ''
    try:
        opts, args = getopt.getopt(argv,"h:i:",["help","uuid="])
    except getopt.GetoptError:
        print('Lifecycle_Monitor_XGBOOST.py --uuid <modelID>')
        sys.exit(2)
    for opt, arg in opts:
        if opt in ("-h", "--help"):
            print('Lifecycle_Monitor_XGBOOST.py --uuid <modelID>')
            sys.exit()
        elif opt in ("-i", "--uuid"):
            uuid = arg
            print('UUID',arg)
    #print('Model ID = ', uuid)

if __name__ == "__main__":
    main(sys.argv[3:])

# Instead use communication via file
file=open(root_path + "output/uuid.txt","r+")
uuid=file.readline()
file.close()
print("UUID = ", uuid)

Number of arguments: 3 arguments.
Argument List: ['/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/home/sasdemo01/.local/share/jupyter/runtime/kernel-994b1b53-b264-42e9-b286-9ea066a643df.json']
Model ID =  


In [3]:
sess = CAS('localhost', 5570, authinfo='/home/sasdemo01/.authinfo', caslib="public")
print(sess)

CAS('localhost', 5570, 'sasdemo01', protocol='cas', name='py-session-1', session='ece4d43c-0bad-0f46-ad7d-8f67ec3b2660')


In [4]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [5]:
data_sets = ['HMEQMONITOR_1_Q1', 'HMEQMONITOR_2_Q2', 'HMEQMONITOR_3_Q3', 'HMEQMONITOR_4_Q4']

targetvar = ['BAD']
inputvars = ['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']

# Dictionaries for different monitoring data sets (d) and to be scored parts X. 
d = {}
X = {}
for idx, ds in enumerate(data_sets):
    print(idx, ds)
    d[idx] = pd.read_csv(datadir + ds + ".csv.zip", compression='zip', na_values=['NULL'])
    Y = d[idx][targetvar]
    X0 = d[idx][inputvars]
    # impute the missing value
    X[idx] = DataFrameImputer().fit_transform(X0)

0 HMEQMONITOR_1_Q1
1 HMEQMONITOR_2_Q2
2 HMEQMONITOR_3_Q3
3 HMEQMONITOR_4_Q4


### Import the Python XGBOOST Model

In [6]:
import pickle
loaded_model = pickle.load(open(datadir + "PythonModelXGBOOST/train/xgboost_model_obj.pkl", "rb"))

### Score New Data in Python

In [7]:
ds_names={}
for idx, ds in enumerate(data_sets):
    y_pred=loaded_model.predict_proba(X[idx])
    temp1=pd.DataFrame(y_pred,columns=["P_BAD0","P_BAD1"])
    frames=[d[idx],temp1]
    ds_names[ds] = pd.concat(frames,axis=1,ignore_index=False)

In [8]:
for idx, ds in enumerate(data_sets):
    results = sess.table.tableExists(caslib="Public", name="{}".format(ds + "_" + uuid))
#   Table not loaded yet
    if(results.exists == 0):
        sess.upload_frame(ds_names[ds],casout={"caslib":"Public","promote":True, "name":"{}".format(ds + "_" + uuid)})

In [9]:
sess.terminate()